In [ ]:
% setup environment
% clear data and number format
clear; format short e
% set path to equilibrium solver
txt=pwd; n=length(txt); rootpath=txt;
n=strfind(rootpath,"USERS"); rootpath=txt(1:n-2);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")
%delete existing phreeqc files
system("rm DATABASE.dat");
system("rm runfile.txt");
%create empty phreeqc files
system("touch DATABASE.dat");
system("touch runfile.txt");

# RESULT OF TESTING

**for tableau**

seems that only the logX with analytical derivatives works best.  I did find some situations that didn't converge.  so I added in some code to make more acidic and more oxic and step towards the answer that way.  doesn't seem to work but the idea is ok.  should check how I do this in the source search file returnspeciationRE or whatever it is called.

X based search didn't work though.  that's fine.  I could probably revise the search method but I like the log one better anyway.

**for PHREEQC**

error seem really big. 1e-6 error for hhigh pH and oxic case.

In [ ]:
%%Note from CS to Scott: in your tableau, I was like: Hfos+ ; should be Hfow-, that generated big errors in PHREEQC because PHREEQC doesn't recognize Hfos+

In [ ]:
% define equilibrium problem

%FeT=1e-5; PT=1e-4; NaT=3*PT; ClT=3*FeT; T=25; HFOsi=1e-30; HFOwi=1e-30; ASFw=1; ASFs=1; 
%pH=2:0.2:11; 
FeT=1e-5/0.997; PT=1e-5/0.997; NaT=3*PT; ClT=3*FeT; T=25; HFOsi=1e-30; HFOwi=1e-30; ASFw=0.2; ASFs=0.005; %/0.997 to mmol/kg at 25C
pH=2:0.2:12; 


% set redox level ------------------------
%peINT=20.75; %oxic
%peINT=15.75; %less oxic
%peINT=10.75; %getting reducing
peINT=5.75; %pretty reducing
%peINT=0; % maximum reducing

pe=peINT-pH;% oxic end


In [ ]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry
flag0=1; %choose solver. 0 for tableau and 1 for phreeqc-based solver
flag1=2; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=1; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=1; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

TOTALS=[FeT; PT; ClT; NaT; HFOsi; HFOwi]';
TOTALS(TOTALS==0)=1e-16; % get rid of zero values, b/c div by zero error

for i=1:length(pH)
    flag5=0; % flags ignored for PHREEQC anyway.
    %if i>1; flag5=1; end
    [Fep3(i),HFO(i),HfowHPO4m(i),MASSERR(i)]=FeHPO4tableau(pH(i),pe(i),TOTALS',flag0,flag1,flag2,flag3,flag4,flag5); 
    if HFO(i)>0; % should make loop until HFO value converges, but from previous tests it converges fast.  one step was how I did the 2008 WER paper
        HFOs=ASFs*HFO(i); HFOw=ASFw*HFO(i); 
        TOTALS=[FeT; PT; ClT; NaT; HFOs; HFOw]';
        [Fep3(i),HFO(i),HfowHPO4m3(i),MASSERR(i)]=FeHPO4tableau(pH(i),pe(i),TOTALS',flag0,flag1,flag2,flag3,flag4,flag5);
    end
    dislay=[pH(i) Fep3(i) HFO(i) HfowHPO4m(i) MASSERR(i)]
end

In [ ]:

plot(pH,Fep3,pH,HFO,pH,HfowHPO4m)
xlabel('pH'); ylabel('[species]'); legend('Fe^{3+}','HFO','HFO-PO4','location','eastoutside')



In [ ]:
plot(pH,MASSERR)

In [ ]:
% === GUARDAR RESULTADOS DE ESTA SESIÓN ==================
quickRun = struct( ...
    "pH",          pH(:),        ...
    "Fe3_aq",      Fep3(:),      ...
    "FeOH3s",      HFO(:),       ...
    "HFO_P_ads",   HfowHPO4m(:), ...
    "MassErr",     MASSERR(:)    ...
);
save("-v7", "phreeqc_runs.mat", "quickRun");
printf("\nquickRun guardado con los datos de la sesión rápida.\n");

In [ ]:
%% ==============================================================
%% 1. Helper robusto: extrae el CPU-time de out.txt
%% ==============================================================
function t = get_cpu_time (fname)
  t = NaN;
  fid = fopen (fname,"rb");
  if fid < 0, warning ("No pude abrir %s", fname); return; end
  data = fread (fid, Inf, "uint8=>uint8");  fclose (fid);
  data(data>127) = 32;                       % limpia UTF-8 raro
  str = char (data');
  tok = regexp (str,"after\\s+([0-9.]+(?:[eE][+-]?[0-9]+)?)\\s+Seconds",
                "tokens","once");
  if !isempty(tok), t = str2double(tok{1}); end
endfunction


%% ==============================================================
%% 2. Bucle principal — resultados + tiempos
%% ==============================================================
if exist("phreeqc_runs_times.mat","file"), delete("phreeqc_runs_times.mat"); endif
nRuns = 50;
save("-v7","phreeqc_runs_times.mat","nRuns");          % cabecera mínima

nPts = numel(pH);

% ===== matrices para TODAS las corridas =====
Fep3        = NaN(nPts,nRuns);   % Fe3+ (aq)
HFO         = NaN(nPts,nRuns);   % Fe(OH)3(s)
HFO_PO4     = NaN(nPts,nRuns);   % HfowHPO4m  (P adsorbido)
MassErr     = NaN(nPts,nRuns);   % error global de masa

t_wall = NaN(nRuns,1);
t_cpu  = NaN(nRuns,1);

for run = 1:nRuns
  tic();
  for i = 1:nPts
    flag5 = 0;
    [Fep3(i,run), HFO(i,run), HFO_PO4(i,run), MassErr(i,run)] = ...
        FeHPO4tableau(pH(i), pe(i), TOTALS', ...
                      flag0, flag1, flag2, flag3, flag4, flag5);

    if HFO(i,run) > 0                         % 2ª pasada si hay HFO
      HFOs   = ASFs * HFO(i,run);
      HFOw   = ASFw * HFO(i,run);
      TOTALS = [FeT; PT; ClT; NaT; HFOs; HFOw]';
      [Fep3(i,run), HFO(i,run), HFO_PO4(i,run), MassErr(i,run)] = ...
          FeHPO4tableau(pH(i), pe(i), TOTALS', ...
                        flag0, flag1, flag2, flag3, flag4, flag5);
    endif
  endfor

  t_wall(run) = toc();
  t_cpu(run)  = get_cpu_time("out.txt");

  % ---------- guardado especial en la 1ª corrida ----------
  if run == 1
    firstRun = struct( ...
        "pH",        pH(:),      ...
        "FeOH3s",    HFO(:,1),   ...
        "Fe3_aq",    Fep3(:,1),  ...
        "HFO_P_ads", HFO_PO4(:,1) );
    MassErr_firstRun = MassErr(:,1);

    save("-v7","-append","phreeqc_runs_times.mat", ...
         "firstRun","MassErr_firstRun");
    printf("\nGuardado firstRun + MassErr_firstRun tras run = 1.\n");
  endif

  printf("Run %02d | wall = %.3f s | cpu = %.3f s  -> guardado\n",
         run, t_wall(run), t_cpu(run));

  % ---- guardado incremental: solo tiempos + run ----
  save("-v7","-append","phreeqc_runs_times.mat","t_wall","t_cpu","run");
  fflush(stdout);
endfor


%% ==============================================================
%% 3. Estadísticos finales y cierre
%% ==============================================================
valid_wall = t_wall(~isnan(t_wall));
valid_cpu  = t_cpu(~isnan(t_cpu));

stats.wall_mean = mean(valid_wall);
stats.wall_std  = std (valid_wall);
stats.cpu_mean  = mean(valid_cpu);
stats.cpu_std   = std (valid_cpu);

save("-v7","-append","phreeqc_runs_times.mat","stats");

printf("\n>> ¡Todo completo! firstRun, tiempos y estadísticos guardados.\n");
